In [ ]:
import pandas as pd
import datetime
import os
import timeit
from ortools.linear_solver import pywraplp

# Load Excel file
file_name = os.fsdecode("C:/Users/Administrator/VSCODE/BatteryOptimization/Data/Dataset_Optimization.xlsx")
workbook = pd.ExcelFile(file_name)

# Ensure output folder exists
output_folder = "output"
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)

# Load timeseries data
marketDF = workbook.parse("Timeseries data")
marketDF = marketDF.iloc[:, :4]
marketDF.columns = ["time", "solar", "load", "market_price_1"]
marketDF = marketDF[~pd.isnull(marketDF["time"])].fillna(0)
marketDF.sort_values(by=["time"], inplace=True)
marketDF.set_index("time", inplace=True)

# Load battery data
battDF = workbook.parse("Battery")
battDF = battDF.iloc[:, :8]
battDF.columns = ["max_charge_rate", "max_discharge_rate", "capacity", "charge_eff", "discharge_eff", "min_soc", "max_soc", "initial_soc"]

# Convert dataframes to dictionaries
marketDict = marketDF.to_dict()
battDict = battDF.to_dict()

# Compute time step
timeIndex = marketDF.index
timeInterval = timeIndex[1] - timeIndex[0]

# Set up input structure
input = type("input", (dict,), {})()
input.update({
    "simData": {
        "startTime": timeIndex[0],
        "dt": int(round(timeInterval.total_seconds())) / 3600,  # in hours
        "tIndex": len(timeIndex)
    },
    "market": {
        key: {
            sub_key: sub_item for sub_key, sub_item in marketDict[key].items()
        } for key in marketDict.keys()
    },
    "batt": {
        key: item[0] for key, item in battDict.items()
    }
})

# Create the MIP solver
solver = pywraplp.Solver.CreateSolver("CBC")
inf = solver.infinity()

tIndex = input["simData"]["tIndex"]
dt = input["simData"]["dt"]
startTime = input["simData"]["startTime"]

# Create datetime list
timestamp = [startTime + i * datetime.timedelta(hours=dt) for i in range(tIndex)]
time = [timestamp[i].strftime("%d/%m/%Y %H:%M") for i in range(len(timestamp))]
time_s = timeit.default_timer()
# Define variables
vGrid = [solver.NumVar(lb=-inf, ub=inf, name=f"vGrid_{i}") for i in range(tIndex)]
vBattPower = [solver.NumVar(lb=-inf, ub=inf, name=f"vBattPower_{i}") for i in range(tIndex)]
vCharge = [solver.NumVar(lb=-inf, ub=0, name=f"vCharge_{i}") for i in range(tIndex)]
vDischarge = [solver.NumVar(lb=0, ub=inf, name=f"vDischarge_{i}") for i in range(tIndex)]
vChargeStatus = [solver.BoolVar(name=f"vChargeStatus_{i}") for i in range(tIndex)]
vSOC = [solver.NumVar(lb=0, ub=1, name=f"vSOC_{i}") for i in range(tIndex)]

# Add constraints
for i in range(tIndex):
    t = timestamp[i]
    solver.Add(vGrid[i] == input["market"]["load"][t] - input["market"]["solar"][t] - vBattPower[i])
    # Battery constraints
    solver.Add(vBattPower[i] == vCharge[i] + vDischarge[i])
    solver.Add(vCharge[i] >= -input["batt"]["max_charge_rate"] * vChargeStatus[i])
    solver.Add(vDischarge[i] <= input["batt"]["max_discharge_rate"] * (1 - vChargeStatus[i]))

    if i == 0:
        solver.Add(vSOC[i] == input["batt"]["initial_soc"] - dt / input["batt"]["capacity"] *
                   (vCharge[i] * (1 - input["batt"]["charge_eff"]) + vDischarge[i] / (1 - input["batt"]["discharge_eff"])))
    else:
        solver.Add(vSOC[i] == vSOC[i-1] - dt / input["batt"]["capacity"] *
                   (vCharge[i] * (1 - input["batt"]["charge_eff"]) + vDischarge[i] / (1 - input["batt"]["discharge_eff"])))

    solver.Add(vSOC[i] >= input["batt"]["min_soc"])
    solver.Add(vSOC[i] <= input["batt"]["max_soc"])

# Objective function
obj = solver.Sum([vGrid[i] * input["market"]["market_price_1"][timestamp[i]] * dt for i in range(tIndex)])
solver.Minimize(obj)

status = solver.Solve()

time_e = timeit.default_timer()
runTime = round(time_e - time_s, 4)

if status == solver.OPTIMAL or status == solver.FEASIBLE:
    print("Solution is found.")
    print("Number of variables =", solver.NumVariables())
    print("Number of constraints =", solver.NumConstraints())
    print("Computation time = ", runTime)
    
    # Extract solution values
    excelWriter = pd.ExcelWriter('output/Result.xlsx', engine='xlsxwriter')
    
    objValue = round(solver.Objective().Value() / 100, 2)
    
    objValueDF = pd.DataFrame.from_dict({"obj_value": objValue}, orient="index", columns=["Total Cost of Importing Power ($)"])
    
    result = list(zip([round(vGrid[i].solution_value(), 4) for i in range(tIndex)], 
                      [round(vBattPower[i].solution_value(), 4) for i in range(tIndex)],
                      [round(vCharge[i].solution_value(), 4) for i in range(tIndex)],
                      [round(vDischarge[i].solution_value(), 4) for i in range(tIndex)],
                      [round(vSOC[i].solution_value(), 4) for i in range(tIndex)],
                      [int(vChargeStatus[i].solution_value()) for i in range(tIndex)]
                      ))
    resultDF = pd.DataFrame(result, index=time, columns=["Grid Power Flow (kW)", "Battery Output (kW)", "Charging Power (kW)", "Discharging Power (kW)", "State-of-charge (SOC)", "Charge Status"])
    
    objValueDF.to_excel(excelWriter, sheet_name='Cost')
    resultDF.to_excel(excelWriter, sheet_name='Operation')
    excelWriter.close()
else:
    print("Solution cannot be found.")

Solution cannot be found.
